In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from numpy import array, unique, array_equal
max_fatures = 1000


####
#Windows instructions
#
# open anaconda terminal as admin
# type in (or copy paste)
#
#    conda create -n py36 python=3.6 anaconda

#   Hit y, then enter, etc
## DO NOT STOP THIS PROCESS


# then type

# activate py36

# then type

# conda install -c conda-forge spacy
# conda install -c conda-forge tensorflow 
# conda install -c conda-forge keras 

# hitting [y] key and enter as required
def load_data():
    print("Loading Data...")
    PrimaryEmotion = pd.read_csv('emotion.xls.csv')
    print("Data Loaded")
    return (PrimaryEmotion['sentence'], array(PrimaryEmotion['emotion']))



def sort_to_2_emotions(sentence_list, emotion_list):
    sorted_list = []
    sorted_emo = []
    for (data, emo) in zip(sentence_list, emotion_list):
        if (emo == 'Joy' or emo == 'Love' or emo == 'Optimism' or emo == 'Awe' or emo == 'Trust'):
            sorted_list.append(data)
            sorted_emo.append("positive")
        if (emo == 'Anger' or emo == 'Disgust' or emo == 'Sadness' or emo == 'Aggression' or emo == 'Contempt' or emo == 'Disapproval' or emo == 'Remorse'):
            sorted_list.append(data)
            sorted_emo.append("negative")
    return (sorted_list, sorted_emo)



def tokenize(sentences):
    print("Tokenizing...")
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(sentences)
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X)
    print("Tokenization Completed")
    return X



def train(tokened_sentences, emotion_list):
    print("Training...")
    embed_dim = 128
    lstm_out = 196
    test_percent=.20

    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = tokened_sentences.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    
    # len(set(emotion_list)) is a hacky way of geting the number of unique elements
    # in a regualar python list (non-numpy)
    model.add(Dense(unique(emotion_list).size,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    #print(model.summary())

    Y = pd.get_dummies(emotion_list).values
    X_train, X_test, Y_train, Y_test = train_test_split(tokened_sentences,Y, test_size = test_percent, random_state = 42)
    #print(X_train.shape,Y_train.shape)
    #print(X_test.shape,Y_test.shape)

    batch_size = 32
    model.fit(X_train, Y_train, epochs = 100, batch_size=batch_size, verbose = 1)
    print("Training Completed")
    print("Testing Against Control... (% of the data) ", test_percent)
    score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("Score   :", score)
    print("Accuracy:", acc)
    return (model, X_test)

### When Splitting data (train_test_split), we don't retain where in the 
### origional set the data is located, thus it takes
### a little trickery to see the results while while only testing against untrained data

def test(model, X_test, tokenized_data, sentence_list, emotion_list):
    predictions = model.predict(tokenized_data, batch_size=32)
    error = 0
    for i in range(len(predictions)):
     
        #  "If the current tokenized data array is in X_test (untrained tokenized arrays)
        
        if(any(array_equal(tokenized_data[i], x) for x in X_test)):
            pos = list(predictions[i]).index(max(predictions[i]))
            print("\n\n")
            print(sentence_list[i])
            print("\nPredicted emotion: ", unique(emotion_list)[pos])
            print("Actual emotion   : ", emotion_list[i])
            if (unique(emotion_list)[pos] != emotion_list[i]):
                error += 1
    print("%  Accuracy when against untrained set: ", 1- (float(error) / len(X_test)))

    
    
#################################
#############MAIN################
#################################
#   jupyter notebook is weird   #




(sentence_list, emotion_list) = load_data()


### Comment the Below line for all 18 emotions. This sorts into "positive" and "negative"
sentence_list, emotion_list = sort_to_2_emotions(sentence_list, emotion_list)

tokenized_data = tokenize(sentence_list)

(model, X_test) = train(tokenized_data, emotion_list)

test(model, X_test, tokenized_data, sentence_list, emotion_list)


Loading Data...
Data Loaded
Tokenizing...
Tokenization Completed
Training...
Epoch 1/100
1128/1128 [==============================] - 7s 7ms/step - loss: 0.6760 - acc: 0.5824
Epoch 2/100
1128/1128 [==============================] - 6s 5ms/step - loss: 0.6246 - acc: 0.6410
Epoch 3/100
1128/1128 [==============================] - 6s 5ms/step - loss: 0.4824 - acc: 0.7934
Epoch 4/100
1128/1128 [==============================] - 6s 5ms/step - loss: 0.3763 - acc: 0.8422
Epoch 5/100
1128/1128 [==============================] - 6s 5ms/step - loss: 0.2836 - acc: 0.8723
Epoch 6/100
1128/1128 [==============================] - 6s 5ms/step - loss: 0.2354 - acc: 0.9034
Epoch 7/100
1128/1128 [==============================] - 6s 5ms/step - loss: 0.1728 - acc: 0.9415
Epoch 8/100
1128/1128 [==============================] - 5s 4ms/step - loss: 0.1342 - acc: 0.9530
Epoch 9/100
1128/1128 [==============================] - 5s 5ms/step - loss: 0.1049 - acc: 0.9610
Epoch 10/100
1128/1128 [=================

1128/1128 [==============================] - 5s 4ms/step - loss: 0.0026 - acc: 1.0000
Epoch 84/100
1128/1128 [==============================] - 5s 4ms/step - loss: 0.0029 - acc: 0.9991
Epoch 85/100
1128/1128 [==============================] - 5s 5ms/step - loss: 0.0017 - acc: 1.0000
Epoch 86/100
1128/1128 [==============================] - 5s 5ms/step - loss: 0.0018 - acc: 1.0000
Epoch 87/100
1128/1128 [==============================] - 5s 4ms/step - loss: 0.0057 - acc: 0.9991
Epoch 88/100
1128/1128 [==============================] - 5s 4ms/step - loss: 0.0101 - acc: 0.9973
Epoch 89/100
1128/1128 [==============================] - 5s 4ms/step - loss: 0.0480 - acc: 0.9876
Epoch 90/100
1128/1128 [==============================] - 5s 4ms/step - loss: 0.0161 - acc: 0.9956
Epoch 91/100
1128/1128 [==============================] - 5s 4ms/step - loss: 0.0077 - acc: 0.9973
Epoch 92/100
1128/1128 [==============================] - 5s 4ms/step - loss: 0.0185 - acc: 0.9947
Epoch 93/100
1128/1128 




For Pete's sake woman, get a grip on yourself.

Predicted emotion:  negative
Actual emotion   :  negative



People with whom one has had personal or professional differences a row or a clash of principle are best avoided because meeting them might demand emotional resources that aren't there, open old wounds, or challenge one's public certainties that in fact feel all too precarious.

Predicted emotion:  negative
Actual emotion   :  negative



His uncle had mentioned every Anglo-Irish family in the district, hoping, he suspected, that he would meet some girl and take a fancy to her.

Predicted emotion:  negative
Actual emotion   :  positive



You're full of the joys of spring at the moment, so make the most of this harmonious period.

Predicted emotion:  positive
Actual emotion   :  positive



You will never look back.

Predicted emotion:  negative
Actual emotion   :  positive



She was in seventh heaven.

Predicted emotion:  positive
Actual emotion   :  positive



Anyway, she


Predicted emotion:  positive
Actual emotion   :  positive



I sleep like a log, a clear conscience.

Predicted emotion:  negative
Actual emotion   :  positive



His government's free-the-market monetarism was beginning to bear fruit.

Predicted emotion:  negative
Actual emotion   :  positive



Evans made the saving tackle, just as he did when Tony Stanger, having barn-stormed out of Wayne Proctor's tackle, threatened to bring the house down in a move created by Craig Chalmers' adroit collection of Mike Rayer's kick, the irrepressible Armstrong arcing out of harm's way on a touchline blast and the thumping support of Cronin and Kenny Milne (one tighthead to the good).

Predicted emotion:  positive
Actual emotion   :  positive



One of Britain's biggest service companies has come to grief with disastrous figures and a crash in its shares.

Predicted emotion:  negative
Actual emotion   :  negative



The rest was all downhill.

Predicted emotion:  negative
Actual emotion   :  negativ




I'm not at my best in red, so you could say I was not on top of the world.

Predicted emotion:  positive
Actual emotion   :  negative



I'd hesitate to give much weight to her story in the normal run because she does seem to have a screw loose.

Predicted emotion:  negative
Actual emotion   :  negative



It hardly needs to be said that Tories are capitalists, and therefore to a woman subscribe to the view that the dominance of heterosexuality, notably the White Heterosexual Family, is to be preserved come hell or high water.

Predicted emotion:  negative
Actual emotion   :  negative



David Walsh argues that Ireland can no longer rely on fire and fury to paper over the cracks in a country where there are fifteen gaelic footballers for every rugby player.

Predicted emotion:  positive
Actual emotion   :  negative



You can not be tentative or apprehensive in your movements; you have to throw caution to the wind and' attack' a manoeuvre with the objective clearly in your mind.

Pr


Predicted emotion:  negative
Actual emotion   :  negative



And so we all had a close shave once a term.

Predicted emotion:  positive
Actual emotion   :  positive



I'll show you the way, watch your step, it's a bit of a difficult climb.

Predicted emotion:  negative
Actual emotion   :  positive



But it's all downhill from here.

Predicted emotion:  negative
Actual emotion   :  negative



It may pass muster as television commentary but on the page its studied neutrality rings false.

Predicted emotion:  negative
Actual emotion   :  negative



McEnroe still my smash hit says Tatum.

Predicted emotion:  positive
Actual emotion   :  positive



Suffice to say Kennel Club standards will be out the window on Top Dog Day.

Predicted emotion:  negative
Actual emotion   :  negative



The way he gained the word that sustains the weary was not like a bolt from the blue, but through patiently listening to God morning by morning.

Predicted emotion:  positive
Actual emotion   :  positive
